In [10]:
import spacy
from spacy.training import Example
from data.dsa_train_data_fixed import TRAIN_DATA

# 1️⃣ Deutsches Basismodell laden
nlp = spacy.load("de_core_news_sm")

# 2️⃣ POS-Tagger-Komponente holen
tagger = nlp.get_pipe("tagger")

"""# 3️⃣ Falls du eigene Tags nutzt, hier hinzufügen (optional)
for sent, ann in TRAIN_DATA:
    for tag in ann["pos"]:
        if tag not in tagger.labels:
            tagger.add_label(tag)"""

# 4️⃣ Fine-Tuning
print("Starte Fine-Tuning auf DSA-Daten ...")
for epoch in range(10):
    losses = {}
    for text, annotations in TRAIN_DATA:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        nlp.update([example], losses=losses)
    print(f"Epoche {epoch+1} - Loss: {losses}")

# 5️⃣ Modell speichern
output_dir = "de_dsa_tagger"
nlp.to_disk(output_dir)
print(f"Modell gespeichert unter: {output_dir}")


Starte Fine-Tuning auf DSA-Daten ...
Epoche 1 - Loss: {'tok2vec': 161.5016918182373, 'tagger': 0.0, 'morphologizer': 53.240132331848145, 'parser': 0.0, 'lemmatizer': 0.0, 'ner': 0.0}
Epoche 2 - Loss: {'tok2vec': 30.115575637901202, 'tagger': 0.0, 'morphologizer': 8.026043093879707, 'parser': 0.0, 'lemmatizer': 0.0, 'ner': 0.0}
Epoche 3 - Loss: {'tok2vec': 4.158587197240422, 'tagger': 0.0, 'morphologizer': 0.7085235720361887, 'parser': 0.0, 'lemmatizer': 0.0, 'ner': 0.0}
Epoche 4 - Loss: {'tok2vec': 0.0064903936869644685, 'tagger': 0.0, 'morphologizer': 0.0011465696719241691, 'parser': 0.0, 'lemmatizer': 0.0, 'ner': 0.0}
Epoche 5 - Loss: {'tok2vec': 0.0003373495041086201, 'tagger': 0.0, 'morphologizer': 5.9781852585920336e-05, 'parser': 0.0, 'lemmatizer': 0.0, 'ner': 0.0}
Epoche 6 - Loss: {'tok2vec': 5.274267037289407e-06, 'tagger': 0.0, 'morphologizer': 1.4538145260327776e-06, 'parser': 0.0, 'lemmatizer': 0.0, 'ner': 0.0}
Epoche 7 - Loss: {'tok2vec': 1.5635855214832933e-06, 'tagger': 0

In [8]:
import spacy
from data.dsa_train_data import TRAIN_DATA

nlp = spacy.blank("de")

for text, ann in TRAIN_DATA:
    doc = nlp.make_doc(text)
    if len(doc) != len(ann["pos"]):
        print(f"⚠️  Tokenanzahl passt nicht: '{text}'")
        print(f"  Tokens: {[t.text for t in doc]}")
        print(f"  POS:    {ann['pos']}")


⚠️  Tokenanzahl passt nicht: 'Der Perainegeweihte heilt die Wunden des Bauern.'
  Tokens: ['Der', 'Perainegeweihte', 'heilt', 'die', 'Wunden', 'des', 'Bauern', '.']
  POS:    ['DET', 'NOUN', 'VERB', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'PUNCT']


In [9]:
import spacy
from data.dsa_train_data import TRAIN_DATA
from pprint import pprint

# ⚙️ Lade ein deutsches Tokenizer-Modell (nur zum Tokenisieren!)
nlp = spacy.blank("de")

print("🔍 Überprüfe DSA-Trainingsdaten auf Token/POS-Längen...\n")

fixed_data = []
errors = 0

for text, ann in TRAIN_DATA:
    doc = nlp.make_doc(text)
    tokens = [t.text for t in doc]
    pos = ann["pos"]

    if len(tokens) != len(pos):
        errors += 1
        print(f"⚠️  Mismatch in Satz: {text}")
        print(f"   Tokens ({len(tokens)}): {tokens}")
        print(f"   POS    ({len(pos)}): {pos}")

        # Vorschlag: gleiche Länge auffüllen
        # Wenn mehr Tokens → kürze POS
        # Wenn weniger Tokens → füge 'X' hinzu (unbekannt)
        if len(tokens) > len(pos):
            corrected_pos = pos + ["X"] * (len(tokens) - len(pos))
        else:
            corrected_pos = pos[:len(tokens)]

        print(f"👉 Vorschlag korrigiert ({len(corrected_pos)}): {corrected_pos}\n")

        fixed_data.append((text, {"pos": corrected_pos}))
    else:
        fixed_data.append((text, ann))

print(f"\n✅ Überprüfung abgeschlossen. {errors} fehlerhafte Sätze gefunden.\n")

# Optional: korrigierte Datei speichern
save_choice = input("💾 Korrigierte Daten als neue Datei speichern? (j/n): ").strip().lower()
if save_choice == "j":
    with open("data/dsa_train_data_fixed.py", "w", encoding="utf8") as f:
        f.write("TRAIN_DATA = [\n")
        for text, ann in fixed_data:
            f.write(f'    ({text!r}, {ann!r}),\n')
        f.write("]\n")
    print("📁 Gespeichert als: data/dsa_train_data_fixed.py")
else:
    print("❌ Keine Datei gespeichert. Nur überprüft.")


🔍 Überprüfe DSA-Trainingsdaten auf Token/POS-Längen...

⚠️  Mismatch in Satz: Der Perainegeweihte heilt die Wunden des Bauern.
   Tokens (8): ['Der', 'Perainegeweihte', 'heilt', 'die', 'Wunden', 'des', 'Bauern', '.']
   POS    (9): ['DET', 'NOUN', 'VERB', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'PUNCT']
👉 Vorschlag korrigiert (8): ['DET', 'NOUN', 'VERB', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN']


✅ Überprüfung abgeschlossen. 1 fehlerhafte Sätze gefunden.

📁 Gespeichert als: data/dsa_train_data_fixed.py


### Wenn du eigene neue Tags brauchst (z. B. „ARTEFAKT“, „GOTT“ usw.)

Dann kannst du nicht das fertige Modell de_core_news_sm nehmen,
sondern musst eine leere Pipeline mit neuem Tagger aufbauen, z. B.:

In [4]:
import spacy
from spacy.pipeline import Tagger
from spacy.training import Example
from data.dsa_train_data import TRAIN_DATA

# 1️⃣ Leere deutsche Pipeline erstellen
nlp = spacy.blank("de")

# 2️⃣ Neuen Tagger hinzufügen
tagger = nlp.add_pipe("tagger")

# 3️⃣ Alle POS-Tags hinzufügen
for _, ann in TRAIN_DATA:
    for tag in ann["pos"]:
        tagger.add_label(tag)

# 4️⃣ Training starten
nlp.begin_training()
for epoch in range(20):
    losses = {}
    for text, annotations in TRAIN_DATA:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        nlp.update([example], losses=losses)
    print(f"Epoche {epoch+1} - Loss: {losses}")

# 5️⃣ Modell speichern
#nlp.to_disk("de_dsa_tagger_custom")
print("✅ Neues DSA-Tagger-Modell gespeichert!")


ValueError: [E971] Found incompatible lengths in `Doc.from_array`: 9 for the array and 8 for the Doc itself.

## Evaluate DSA POS Tagger

In [2]:
import spacy
from data.dsa_train_data import TRAIN_DATA

# Lade beide Modelle
nlp_base = spacy.load("de_core_news_sm")
nlp_dsa  = spacy.load("de_dsa_tagger")

print("🔍 Vergleiche Basismodell vs. DSA-Modell\n")

# Hilfsfunktion für Genauigkeit
def evaluate(model, data):
    correct = 0
    total = 0
    for text, annotations in data:
        doc = model(text)
        predicted_tags = [token.pos_ for token in doc]
        gold_tags = annotations["pos"]
        total += len(gold_tags)
        correct += sum(p == g for p, g in zip(predicted_tags, gold_tags))
    return correct / total if total > 0 else 0

# 1️⃣ Genauigkeit berechnen
acc_base = evaluate(nlp_base, TRAIN_DATA)
acc_dsa  = evaluate(nlp_dsa,  TRAIN_DATA)

print(f"Basismodell: {acc_base*100:.2f}% korrekt")
print(f"DSA-Modell : {acc_dsa*100:.2f}% korrekt\n")

# 2️⃣ Zeige Unterschiede
print("⚔️  Unterschiede zwischen den Modellen:")
for text, annotations in TRAIN_DATA:
    doc_base = nlp_base(text)
    doc_dsa  = nlp_dsa(text)
    gold_tags = annotations["pos"]
    print(f"\n📜 Satz: {text}")
    for token, gold, pred_b, pred_d in zip(doc_dsa, gold_tags,
                                           [t.pos_ for t in doc_base],
                                           [t.pos_ for t in doc_dsa]):
        mark = "✅" if pred_d == gold else "❌"
        diff = "" if pred_b == pred_d else f"(Base:{pred_b}→DSA:{pred_d})"
        print(f"  {token.text:12} Gold:{gold:6} DSA:{pred_d:6} {mark} {diff}")


OSError: [E050] Can't find model 'de_dsa_tagger'. It doesn't seem to be a Python package or a valid path to a data directory.